In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import needed libraries

In [ ]:
from dataclasses import dataclass, field
from transformers import RobertaForMaskedLM, RobertaTokenizerFast, RobertaModel, DefaultDataCollator, DataCollatorWithPadding, DataCollatorForLanguageModeling, Trainer, LongformerForMaskedLM,  LongformerTokenizerFast
from transformers import TrainingArguments, HfArgumentParser, AutoModelForMaskedLM, AutoModel, AutoTokenizer
from datasets import load_dataset, Dataset
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from accelerate import Accelerator, DeepSpeedPlugin
from torchmetrics import AUROC, AveragePrecision
from transformers import RobertaConfig
from tokenizers.implementations import CharBPETokenizer
from tokenizers.processors import BertProcessing
from safetensors.torch import load_file
import threading
import os
import torch.nn as nn
import logging
import math
import copy
import torch

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

## Data prepocessing

### Download and extract FASTA files

In [ ]:
#!wget -O interaction.protein.gz  "https://rest.uniprot.org/idmapping/uniprotkb/results/stream/b36da80dd9f77ce4d631842f8d85fe80785b0702?compressed=true&format=fasta"
#!gzip -df "result.fasta.gz"

### Create directories for dataset

In [ ]:
!mkdir dataset
!mkdir dataset/train
!mkdir dataset/val
!mkdir dataset/test
!mkdir aminobert

## Pretraining with MLM

### Enable CUDA

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Enable XLA

In [ ]:
#import torch_xla
#import torch_xla.core.xla_model as xm
#device = xm.xla_device()
#print(device)

### Initialize an Argument Parser for transformers.TrainingArguments

In [ ]:
@dataclass
class ModelArgs:
    attention_window: int = field(default=512, metadata={"help": "Size of attention window"})
    max_pos: int = field(default=4096, metadata={"help": "Maximum position"})

parser = HfArgumentParser((TrainingArguments, ModelArgs,))

### Define tokenize function and pretraining and evaluation function

In [ ]:
def tokenize_function(tokenizer, examples):
    return tokenizer(examples["fasta"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

def pretrain_and_evaluate(args, model, tokenizer, eval_only, model_path):
    data_files = {"val": args.val_datapath}
    
    if eval_only:
        data_files["train"] = data_files["val"]
    else:
        logger.info(f'Loading and tokenizing training data is usually slow: {args.train_datapath}')
        data_files["train"] = args.train_datapath

    datasets = load_dataset("csv", data_files=data_files)
    datasets = datasets.map(lambda x: tokenize_function(tokenizer, x), batched=True)

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
    trainer = Trainer(model=model, args=args, data_collator=data_collator,
                      train_dataset=datasets["train"], eval_dataset=datasets["val"],)
        
    print("I'm evaluating...")
    eval_loss = trainer.evaluate()
    eval_loss = eval_loss['eval_loss']
    logger.info(f'Initial eval bpc: {eval_loss/math.log(2)}')

    print(f"Initial eval bpc: {eval_loss/math.log(2)}")

    if not eval_only:
        print("I'm training...")
        trainer.train(model_path=model_path)
        trainer.save_model()

        eval_loss = trainer.evaluate()
        eval_loss = eval_loss['eval_loss']
        logger.info(f'Eval bpc after pretraining: {eval_loss/math.log(2)}')
        print(f"Eval bpc after pretraining: {eval_loss/math.log(2)}")
        

### Define arguments for pretraining (TrainingArguments)
Temporary parameters. In the real model, parameters need to be "optimized", or in any case chosen using appropriate heuristics.<br>
Scaling laws will be used to calculate the optimal amount of training steps/parameters. Learning rate, cosine annealing cycle length and decay and other information on choosing hyperparameters can be found in this paper https://arxiv.org/pdf/2203.15556 and this paper https://openreview.net/pdf?id=Bx6qKuBM2AD.


In [ ]:
training_args, model_args = parser.parse_args_into_dataclasses(look_for_args_file=False, args=[
    '--output_dir', 'tmp',
    '--learning_rate', '2e-4',
    '--weight_decay', '0.01',
    '--adam_epsilon', '1e-6',
    '--max_steps', '5000',
    '--logging_steps', '100',
    '--save_steps', '500',
    '--max_grad_norm', '5.0',
    '--per_device_eval_batch_size', '8',
    '--per_device_train_batch_size', '2',  # 32GB gpu with fp32
    '--gradient_accumulation_steps', '32',
    '--do_train',
    '--do_eval',
    #'--num_train_epochs', '1',
    '--tpu_num_cores', '8',                      # Number of TPU cores (typically 8)
    '--lr_scheduler_type', 'cosine',
    '--warmup_ratio', '0.1',
    # This cosine scheduler drops to min lr rate of zero, not of 10x less the initial lr like in the paper

    #'--lr_scheduler_kwargs', '{"num_cycles": 0.5}',           
    # This drops approximately 10x  
    '--lr_scheduler_kwargs', '{"num_cycles": 0.41}',            

    #'--warmup_steps', '500',

])

training_args.train_datapath = './dataset/train/train_proteins.csv'
training_args.val_datapath = './dataset/val/val_proteins.csv'

training_args.prediction_loss_only = True
print("Device:", training_args.device)


### Define tokenizer and model

In [ ]:
# ------------------------  Train tokenizer on DNA sequences --------------------------
#files = ['./dataset/train/tokenizer_train.txt'] 
#roberta_base_tokenizer = CharBPETokenizer()

# Customize training (change vocab size)
#roberta_base_tokenizer.train(files=files, vocab_size=500, min_frequency=2, special_tokens=[
#    "<s>",
#    "<pad>",
#    "</s>",
#    "<unk>",
#    "<mask>",
#])

#roberta_base_tokenizer._tokenizer.post_processor = BertProcessing(
#    ("</s>", roberta_base_tokenizer.token_to_id("</s>")),
#    ("<s>", roberta_base_tokenizer.token_to_id("<s>")),
#)

#roberta_base_tokenizer.enable_truncation(max_length=512)
#roberta_base_tokenizer.save_model("./aminobert")

# -------------------------------------------------------------------------------------


In [ ]:

# ---------------------  Define model and resize token embeddings ---------------------

roberta_base_tokenizer = RobertaTokenizerFast.from_pretrained("./aminobert")

roberta_base = RobertaForMaskedLM.from_pretrained('roberta-base')
roberta_base.resize_token_embeddings(len(roberta_base_tokenizer))
print(f"Roberta parameters: {int(roberta_base.num_parameters()/1000000)}M")


### Create a smaller version of RoBERTa (for testing) and pretrain it

In [ ]:
def deleteEncodingLayers(model, num_layers_to_keep):  # must pass in the full bert model
    oldModuleList = model.roberta.encoder.layer
    newModuleList = nn.ModuleList()

    # Now iterate over all layers, only keepign only the relevant layers.
    for i in range(0, num_layers_to_keep):
        newModuleList.append(oldModuleList[i])

    # create a copy of the model, modify it with the new list, and return
    copyOfModel = copy.deepcopy(model)
    copyOfModel.roberta.encoder.layer = newModuleList

    return copyOfModel
    


In [ ]:

small_roberta = deleteEncodingLayers(roberta_base,4)
print(f"Small roberta parameters: {int(small_roberta.num_parameters()/1000000)}M")


In [ ]:
pretrain_and_evaluate(training_args, small_roberta, roberta_base_tokenizer, eval_only=False, model_path=None)

In [ ]:
# Assuming `model` is your pretrained model and `tokenizer` is your tokenizer
#output_dir = "./pretrained"

# Save the model
#small_roberta.save_pretrained(output_dir)

# Save the tokenizer
#roberta_base_tokenizer.save_pretrained(output_dir)


## Pretraining with Functional Prediction

## Finetuning

In [ ]:
# Load the model for sequence classification
model_encoder = AutoModelForMaskedLM.from_pretrained("./pretrained")
model_encoder = deleteEncodingLayers(model_encoder,4).roberta

In [ ]:

class InputDataset(Dataset):
    def __init__(self, interactions_file, tokenizer1, tokenizer2):
        interactions = pd.read_csv(interactions_file)
        
        if interactions.shape[1] == 3:
            self.targets = torch.tensor(interactions.iloc[:,2].values, dtype=torch.long)
        elif interactions.shape[1] != 2:
            raise Exception(f"Invalid input file: input should have shape (N, 3) or (N, 2), but has shape {self.interactions.shape}.")
        
        self.inputs = interactions.iloc[:,:2].values
        self.tokenizer1 = tokenizer1
        self.tokenizer2 = tokenizer2


    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        input1 = self.tokenizer1(list(np.atleast_1d(self.inputs[idx, 0])),
                            padding="max_length", 
                            truncation=True, 
                            max_length=512,
                            return_tensors="pt")
        input2 = self.tokenizer2(list(np.atleast_1d(self.inputs[idx, 1])),
                            padding="max_length", 
                            truncation=True, 
                            max_length=512,
                            return_tensors="pt")
        
        input1["input_ids"] = input1["input_ids"].view(-1)
        input2["input_ids"] = input2["input_ids"].view(-1)


        return ((input1, input2), self.targets[idx])


### FFN Interaction Head

In [ ]:
class InteractionModelFFN(nn.Module):
    def __init__(self, encoder, num_labels):
        super().__init__()
        self.encoder = encoder
        self.linear_gelu_stack = nn.Sequential(
            nn.Linear(1536, 1024),
            nn.Dropout(0.2),
            nn.GELU(),
            nn.Linear(1024, 512),
            nn.Dropout(0.2),
            nn.GELU(),
            nn.Linear(512, num_labels),
        ) 
        
    def forward(self, x1, x2):
        
        y1 = self.encoder(**x1).last_hidden_state
        y2 = self.encoder(**x2).last_hidden_state
        z = torch.concatenate([y1.mean(dim=1), y2.mean(dim=1)], dim=1)
        y = self.linear_gelu_stack(z)
        return y


### Multihead Attention Interaction Head

In [ ]:
# The key is the drug because it binds to the target (like the key of a lock)

class InteractionModelATTN(nn.Module):
    def __init__(self, encoder, num_labels, num_heads=1):
        super().__init__()
        self.encoder = encoder
        self.multihead_attention = nn.MultiheadAttention(768, num_heads, dropout = 0.2, batch_first=True)

        self.output = nn.Linear(768, num_labels)
        
        self.conv = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=1)
        self.norm = nn.LayerNorm(768)
        
    def forward(self, x1, x2):
        y1 = self.encoder(**x1).last_hidden_state       # The target in the real case
        y2 = self.encoder(**x2).last_hidden_state       # The drug in the real case
        out, _ = self.multihead_attention(y1, y2, y1)

        out = self.norm(out)
                
        out = torch.squeeze(self.conv(out), axis=1)
        out = self.output(out)
        return out


In [ ]:

tokenizer1 = RobertaTokenizerFast.from_pretrained("./aminobert")
tokenizer2 = RobertaTokenizerFast.from_pretrained("./aminobert")



In [ ]:
res = tokenizer1("ACGCGCAG", max_length=512, padding="max_length", return_tensors="pt")
res1 = tokenizer1(["ACGCGCAG", "DACDAFV"] , max_length=512,  padding="max_length", return_tensors="pt")
res2 = tokenizer2(["ACGCGCCDFAG", "CADCAEDF"] , max_length=512,  padding="max_length", return_tensors="pt")


multihead_attention = nn.MultiheadAttention(768, 1, dropout = 0.2, batch_first=True)
model_encoder.eval()
p1 = model_encoder(**res1).last_hidden_state
p2 = model_encoder(**res2).last_hidden_state

att = multihead_attention(p1, p2, p1)[0]

norm = nn.LayerNorm(768)
out = norm(att)
conv = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=1)
print(list(conv.parameters())[0].shape)
out = torch.squeeze(conv(out), axis=1)
print(out.shape)

#z = torch.concatenate([p1.mean(dim=1), p2.mean(dim=1)], dim=1)

#print(z.shape)
#model_encoder.config


#datasets = datasets.map(lambda x: finetune_tokenize(tokenizer, x), batched=True)
#tokenizer(datasets["train"]["text"][:100])


In [ ]:
import matplotlib.pyplot as plt
train_interactions_file = "dataset/train/train_inters.csv"
val_interactions_file = "dataset/val/val_inters.csv"

train_parameters = {"train_batch_size": 4,
                    "device": "cuda",
                    "learning_rate": 2e-5,
                    "adam_epsilon": 1e-6,
                    "gradient_accumulation_steps": 32,
                    "num_training_steps":3000,
                    "log_performance_every":20,
                    }

train_dataset = InputDataset(train_interactions_file, tokenizer1, tokenizer2)
val_dataset = InputDataset(val_interactions_file, tokenizer1, tokenizer2)

def evaluate_model(cpu_model, data_loader, thread_id):
    cpu_model.eval() 

    auroc_metric = AUROC(task="multiclass", num_classes=2)
    auprc_metric = AveragePrecision(task="multiclass", num_classes=2)
    
    eval_n = 0

    with torch.no_grad():
        for source, targets in data_loader:
      
            if eval_n > 150:
                break
                        
            output = cpu_model(source[0], source[1])
            auroc_metric.update(output, targets)
            auprc_metric.update(output, targets)

            eval_n += 1

    auroc_score = auroc_metric.compute()
    auprc_score = auprc_metric.compute()
    
    print(f"Step {thread_id} - AUROC val: {auroc_score:.4f}, AUPRC val: {auprc_score:.4f}")
      
#add weight decay
def finetune_and_evaluate(model):
    deepspeed_plugin = DeepSpeedPlugin(zero_stage=3, 
                                       offload_param_device = "cpu",
                                        offload_optimizer_device = "cpu"
                                       )
    accelerator = Accelerator(deepspeed_plugin=deepspeed_plugin, gradient_accumulation_steps=train_parameters["gradient_accumulation_steps"])
    device = accelerator.device

    auroc_metric_train = AUROC(task="multiclass", num_classes=2).to(device)
    auprc_metric_train = AveragePrecision(task="multiclass", num_classes=2).to(device)

    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=train_parameters["learning_rate"], weight_decay=0.01)
    train_loader = DataLoader(train_dataset, batch_size=train_parameters["train_batch_size"], shuffle=True)
    val_loader = DataLoader(val_dataset, shuffle=True)
    with torch.no_grad():
        for source, targets in val_loader:
            source1, targets1 = source, targets
            break

    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)
    print("DEVICE USED:", accelerator.device)
    print("NUM PROCESSES:", accelerator.num_processes)

    model.train()

    step = 0  # Initialize the step variable
    epoch = 1
    max_steps = train_parameters["num_training_steps"]
    criterion = torch.nn.CrossEntropyLoss()
    mean_loss = 0

    while step < max_steps:
        print(f"EPOCH {epoch}:")
        for train_source, train_targets in train_loader:
            with accelerator.accumulate(model):
                output = model(train_source[0], train_source[1])
                loss = criterion(output, train_targets)
                #print(loss)
                grad_mean = []
                mean_loss+=loss.item()
                accelerator.backward(loss)
                auroc_metric_train.update(output, train_targets)
                auprc_metric_train.update(output, train_targets)

                # Compute metrics at a frequency of your choice (e.g., every 5 steps)
                if (step + 1) % (train_parameters["log_performance_every"]*train_parameters["gradient_accumulation_steps"]) == 0:
                    # compute metrics for validation set    

                    cpu_model = copy.deepcopy(model).cpu()
                    output = cpu_model(source1[0], source1[1])
                    print(output.detach(), targets1.detach())
                    eval_thread = threading.Thread(
                        target=evaluate_model, 
                        args=(cpu_model, val_loader, step+1)
                    )
                    eval_thread.start()
                    
                    auroc_score_train = auroc_metric_train.compute()
                    auprc_score_train = auprc_metric_train.compute()
                    auroc_metric_train.reset()
                    auroc_metric_train.reset()


            
                optimizer.step()
                
                for param in model.parameters():
                    if param.grad is not None:
                        grad_mean.append(param.grad.mean().cpu().detach())
                optimizer.zero_grad()  # Reset gradients
                   

                if (step + 1) % (train_parameters["log_performance_every"]*train_parameters["gradient_accumulation_steps"]) == 0:
                    mean_loss/= (train_parameters["log_performance_every"]*train_parameters["gradient_accumulation_steps"])
                    print(f"Step {step + 1} - Loss: {mean_loss:.4f}, AUROC: {auroc_score_train:.4f}, AUPRC: {auprc_score_train:.4f}")
                    mean_loss = 0
                    plt.yscale("log")

                    plt.plot(grad_mean)
                    plt.show()

                step += 1

                if step >= max_steps:
                    break
        epoch+=1
       



In [ ]:
#model_encoder1 = AutoModelForMaskedLM.from_pretrained("./pretrained")
#model_encoder1 = deleteEncodingLayers(model_encoder1,4).roberta
#model_encoder2 = AutoModelForMaskedLM.from_pretrained("./pretrained")
#model_encoder2 = deleteEncodingLayers(model_encoder2,4).roberta

model_encoder1 = RobertaForMaskedLM.from_pretrained('roberta-base')
model_encoder1.resize_token_embeddings(len(tokenizer1))
model_encoder1 = deleteEncodingLayers(model_encoder1,2).roberta
model_encoder2 = RobertaForMaskedLM.from_pretrained('roberta-base')
model_encoder2.resize_token_embeddings(len(tokenizer1))
model_encoder2 = deleteEncodingLayers(model_encoder2,2).roberta



models = [InteractionModelFFN(model_encoder1, 2), InteractionModelATTN(model_encoder2,2)][1:]

for model in models:
   finetune_and_evaluate(model)

## Longformer

In [ ]:
longformer = LongformerForMaskedLM.from_pretrained('allenai/longformer-base-4096')
longformer_base_tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')



In [ ]:
pretrain_and_evaluate(training_args, longformer, longformer_base_tokenizer, eval_only=False, model_path=None)


## Feature selection

In [ ]:
train_dataset = pd.read_csv(train_interactions_file)
val_dataset = pd.read_csv(val_interactions_file)


In [ ]:
"""
import optuna
from sklearn.ensemble import RandomForestClassifier

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "min_samples_split": trial.suggest_float("min_samples_split", 0.1, 1),
        "random_state": 14
    }
    train_dataset = pd.read_csv(train_interactions_file)
    val_dataset = pd.read_csv(val_interactions_file)
    
    model = RandomForestClassifier(**params)
    model.fit(train_x, train_targets)

    score_metric = AveragePrecision(task="multiclass", num_classes=2)
    
    # In binary classification, the probability of the class with the “greater label” should be provided. 
    # The “greater label” corresponds to classifier.classes_[1] and thus classifier.predict_proba(X)[:, 1].
    # See https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case
    score = score_metric.update(val_inputs, val_outputs)
    return score
"""


In [ ]:
"""
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=14),
    study_name="random_forest_feature_select",
    load_if_exists=True
)

study.optimize(objective, n_trials=10)
"""